In [1]:
import glob
import os
import pandas as pd


In [2]:
# ファイル名取り出し
file_names = []
for f in glob.glob('input/*.csv'):
    file_names.append(os.path.split(f)[1])
file_names


['input1.csv', 'input2.csv']

In [3]:
# データ読み込み
def read_data():
    years = []
    for i in file_names:
        # # 複数ヘッダのまま読み込み
        # df_head = pd.read_csv('input/' + i, encoding="SHIFT-JIS", header=[0,1,2])
        # years.append(df)

        # ヘッダ読み込み
        temp = pd.read_csv('input/' + i, header=None, nrows=3)
        temp = temp.fillna('')
        header = temp.iloc[0] + '_[' + temp.iloc[1] + ']_' + temp.iloc[2]
        header = header.tolist()
        # '_[]_'を削除
        temp = []
        for j in header:
            if '_[]_' in j:
                j = j.replace('_[]_', '')
            temp.append(j)
        header = temp
        
        # データ読み込み
        df = pd.read_csv('input/' + i, header=None, skiprows=3)
        df.columns = header

        # 結合
        years.append(df)
        df_concat = pd.concat(years, axis=1)
        
    return df_concat

df = read_data()
df.head()

,date,time,1_[input1]_cat1,2_[input1]_cat2,3_[input1]_num1,4_[input1]_num2,date,time,5_[input2]_cat3,6_[input2]_cat4,7_[input2]_num3,8_[input2]_num4
0,2016/4/13,1:00:00,A,A-1,1,10.1,2016/4/13,1:00:00,A,A-1,31,30.1
1,2016/4/13,2:00:00,A,A-1,2,10.2,2016/4/13,2:00:00,A,A-1,32,30.2
2,2016/4/13,3:00:00,A,A-2,3,10.3,2016/4/13,3:00:00,A,A-2,33,30.3
3,2016/4/13,4:00:00,A,A-2,4,10.4,2016/4/13,4:00:00,A,A-2,34,30.4
4,2016/4/13,5:00:00,B,B-1,5,10.5,2016/4/13,5:00:00,B,B-1,35,30.5


In [4]:
# 重複列を削除
df = df.loc[:,~df.columns.duplicated()]
df.head()

,date,time,1_[input1]_cat1,2_[input1]_cat2,3_[input1]_num1,4_[input1]_num2,5_[input2]_cat3,6_[input2]_cat4,7_[input2]_num3,8_[input2]_num4
0,2016/4/13,1:00:00,A,A-1,1,10.1,A,A-1,31,30.1
1,2016/4/13,2:00:00,A,A-1,2,10.2,A,A-1,32,30.2
2,2016/4/13,3:00:00,A,A-2,3,10.3,A,A-2,33,30.3
3,2016/4/13,4:00:00,A,A-2,4,10.4,A,A-2,34,30.4
4,2016/4/13,5:00:00,B,B-1,5,10.5,B,B-1,35,30.5


In [5]:
# 列結合
df['datetime'] = df['date'] + 'T' + df['time']
df = df.drop(columns=['date', 'time'])
df['datetime'] = pd.to_datetime(df['datetime'])
df.head()

,1_[input1]_cat1,2_[input1]_cat2,3_[input1]_num1,4_[input1]_num2,5_[input2]_cat3,6_[input2]_cat4,7_[input2]_num3,8_[input2]_num4,datetime
0,A,A-1,1,10.1,A,A-1,31,30.1,2016-04-13 01:00:00
1,A,A-1,2,10.2,A,A-1,32,30.2,2016-04-13 02:00:00
2,A,A-2,3,10.3,A,A-2,33,30.3,2016-04-13 03:00:00
3,A,A-2,4,10.4,A,A-2,34,30.4,2016-04-13 04:00:00
4,B,B-1,5,10.5,B,B-1,35,30.5,2016-04-13 05:00:00


In [6]:
# 行の絞り込み
df_a = df[df['1_[input1]_cat1'] == 'A']
df_a.head()

,1_[input1]_cat1,2_[input1]_cat2,3_[input1]_num1,4_[input1]_num2,5_[input2]_cat3,6_[input2]_cat4,7_[input2]_num3,8_[input2]_num4,datetime
0,A,A-1,1,10.1,A,A-1,31,30.1,2016-04-13 01:00:00
1,A,A-1,2,10.2,A,A-1,32,30.2,2016-04-13 02:00:00
2,A,A-2,3,10.3,A,A-2,33,30.3,2016-04-13 03:00:00
3,A,A-2,4,10.4,A,A-2,34,30.4,2016-04-13 04:00:00


In [7]:
# 列の絞り込み
temp = []
for i in df_a.columns.tolist():
    if '[input2]'in i:
        temp.append(i)
df_a = df_a[temp]
df_a

,5_[input2]_cat3,6_[input2]_cat4,7_[input2]_num3,8_[input2]_num4
0,A,A-1,31,30.1
1,A,A-1,32,30.2
2,A,A-2,33,30.3
3,A,A-2,34,30.4


In [8]:
# 列の複数条件絞り込み
temp = []
for i in df_a.columns.tolist():
    if '[input2]' in i and 'num3' in i:
        temp.append(i)
df_a = df_a[temp]
df_a

,7_[input2]_num3
0,31
1,32
2,33
3,34
